<a href="https://colab.research.google.com/github/jserrataylor/cursoAI/blob/main/Tipos_de_Redes_Neuronales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Tipos de Redes Neuronales**

Algunos de los tipos más comunes de redes neuronales y sus diferencias principales. Cada tipo de red neuronal es adecuado para diferentes tipos de problemas, basándose en la naturaleza de los datos y el tipo de tarea a realizar. Por ejemplo, las CNNs son ideales para datos visuales, mientras que las RNN y sus variantes son mejores para datos secuenciales como texto o series temporales.

1. **Redes Neuronales Densas o Completamente Conectadas (Fully Connected Neural Networks)**
   - **Descripción**: En estas redes, cada neurona en una capa está conectada a todas las neuronas en la capa siguiente.
   - **Uso**: Son útiles para problemas de clasificación y regresión general, donde la relación entre las entradas y las salidas no depende de datos espaciales o temporales.
   - **Diferencia**: No consideran la relación espacial o temporal entre las entradas.

2. **Redes Neuronales Convolucionales (Convolutional Neural Networks, CNNs)**
   - **Descripción**: Se especializan en procesar datos con una estructura en forma de rejilla, como imágenes.
   - **Uso**: Son ideales para tareas de visión computacional como reconocimiento de imágenes, análisis de video y clasificación de imágenes.
   - **Diferencia**: Utilizan capas convolucionales que aplican filtros para capturar características espaciales.

3. **Redes Neuronales Recurrentes (Recurrent Neural Networks, RNNs)**
   - **Descripción**: Diseñadas para trabajar con secuencias de datos, tienen conexiones recurrentes que permiten información persistente.
   - **Uso**: Se utilizan para tareas como el procesamiento del lenguaje natural, reconocimiento de voz y análisis de series temporales.
   - **Diferencia**: Capaces de manejar secuencias de datos y recordar información previa gracias a sus conexiones recurrentes.

4. **Long Short-Term Memory Networks (LSTM)**
   - **Descripción**: Una variante avanzada de las RNN, diseñada para evitar el problema del desvanecimiento del gradiente al procesar secuencias largas.
   - **Uso**: Útiles en aplicaciones similares a las RNN, pero mejoran el rendimiento en secuencias largas.
   - **Diferencia**: Incorporan celdas especiales que pueden mantener información durante períodos más largos y regular el flujo de información.

5. **Redes Neuronales de Memoria a Corto Plazo (Short-Term Memory Networks, GRU)**
   - **Descripción**: Similar a LSTM, pero con una estructura más simplificada.
   - **Uso**: Procesamiento de secuencias donde se requiere eficiencia computacional.
   - **Diferencia**: Ofrecen un balance entre el poder de modelado de LSTM y la eficiencia de las RNN tradicionales.

6. **Autoencoders**
   - **Descripción**: Redes diseñadas para aprender representaciones codificadas (comprimidas) de los datos de entrada.
   - **Uso**: Utilizados para reducción de dimensionalidad, des-ruido de datos y generación de nuevas instancias de datos.
   - **Diferencia**: Su objetivo es reproducir la entrada en la salida, aprendiendo una representación compacta en el proceso.

7. **Redes Neuronales Generativas Antagónicas (Generative Adversarial Networks, GANs)**
   - **Descripción**: Consisten en dos redes, una generativa y otra discriminativa, que se entrenan en conjunto.
   - **Uso**: Creación de contenido nuevo (como imágenes, música, texto), mejoramiento de imágenes, y más.
   - **Diferencia**: Utilizan un enfoque de aprendizaje adversario donde dos redes se entrenan en competición.

8. **Redes Neuronales de Gráficos (Graph Neural Networks, GNNs)**
   - **Descripción**: Diseñadas para trabajar con datos en forma de gráficos.
   - **Uso**: Análisis de redes sociales, sistemas de recomendación, y análisis de estructuras moleculares.
   - **Diferencia**: Capaces de manejar datos estructurados como gráficos, donde las relaciones entre los nodos son clave.

## **Requerimientos de preinstalación**

In [ ]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install accelerate
!pip install transformers[torch]
!pip install accelerate -U

# **Redes Neuronales - Transformer - Bert**

In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from sklearn.metrics import accuracy_score, recall_score, f1_score

# Generar 100 oraciones sintéticas con etiquetas de sentimiento
data = {
    'sentence': [
        "I love spending time with my family",  # Positivo
        "I feel lost and alone",  # Negativo
        "This beautiful scenery is breathtaking",  # Positivo
        "This has been a terrible day",  # Negativo
        "I feel great today",  # Positivo
        "I'm worried about the future",  # Negativo
        "The movie was a delightful experience",  # Positivo
        "I feel sick and tired",  # Negativo
        "I am so grateful for all the good in my life",  # Positivo
        "I'm disappointed with the results",  # Negativo
        # ... (continuar alternando hasta completar 100)
    ] + ["This is a good day" if i % 2 == 0 else "This is a bad day" for i in range(10, 100)],
    'label': [1 if i % 2 == 0 else 0 for i in range(100)]
}

# Crear DataFrame
df = pd.DataFrame(data)

# Dividir los datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2)
true_labels = test_df['label'].tolist()

# Cargar tokenizer y modelo de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True, max_length=128)

train_encodings = tokenize_function(train_df['sentence'].tolist())
test_encodings = tokenize_function(test_df['sentence'].tolist())

# Clase para el conjunto de datos
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convertir a formato PyTorch
train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
test_dataset = CustomDataset(test_encodings, true_labels)

# Definir argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Entrenar el modelo
trainer.train()

# Hacer predicciones en el conjunto de prueba
predictions = trainer.predict(test_dataset)

# Convertir logits a etiquetas predichas
pred_labels = np.argmax(predictions.predictions, axis=1)

# Cálculo de métricas
accuracy = accuracy_score(true_labels, pred_labels)
recall = recall_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels)

# Imprimir las métricas
print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


Accuracy: 0.9
Recall: 1.0
F1 Score: 0.9090909090909091


In [ ]:
# Nuevas oraciones para predecir
new_sentences = ["I can't continue working here!", "This is a bad day!", "I'm sure this day is sad"]

# Tokenizar las nuevas oraciones
new_encodings = tokenizer(new_sentences, padding="max_length", truncation=True, max_length=128)

# Crear un conjunto de datos para las nuevas oraciones
new_dataset = CustomDataset(new_encodings, [0] * len(new_sentences))  # Etiquetas ficticias

# Hacer predicciones
new_predictions = trainer.predict(new_dataset)

# Convertir logits a etiquetas predichas
new_pred_labels = np.argmax(new_predictions.predictions, axis=1)

# Imprimir las etiquetas predichas
print(new_pred_labels)

[0 0 1]


## **Redes Neuronales - Transformer - GPT-2 de OpenAI**

In [ ]:
# Importar las bibliotecas necesarias
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
import torch

# Cargar tokenizer y modelo de GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Asegurarse de que el tokenizer utilice el mismo token EOS que el modelo GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Tus datos de texto
texts = ["<Había una vez en un pueblo lejano, un pequeño pero valiente guerrero. Este guerrero se embarcó en una aventura épica para salvar a su pueblo de un dragón feroz. Durante su viaje, enfrentó numerosos desafíos, pero nunca perdió la esperanza ni el coraje.>", "<El amanecer en la playa era un espectáculo impresionante. Los colores del cielo cambiaban de tonos suaves de rosa a naranjas brillantes, mientras el sol se elevaba lentamente sobre el horizonte del mar tranquilo. Las olas rompían suavemente en la orilla, creando una melodía calmante.>", "<A menudo me encuentro reflexionando sobre el significado de la vida. Creo que la felicidad se encuentra en las pequeñas cosas: una sonrisa genuina, una conversación sincera, o el cálido abrazo de un ser querido. Estos momentos son los que verdaderamente atesoramos.>"]  # Reemplaza esto con tu conjunto de datos

# Tokenizar los datos
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Clase para el conjunto de datos
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Las etiquetas son las mismas que input_ids para entrenamiento de lenguaje
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.encodings['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Convertir a formato PyTorch
dataset = TextDataset(encodings)

# Definir argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=3,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# Entrenar el modelo
trainer.train()


Step,Training Loss


TrainOutput(global_step=6, training_loss=4.912951151529948, metrics={'train_runtime': 31.3583, 'train_samples_per_second': 0.287, 'train_steps_per_second': 0.191, 'total_flos': 459302400000.0, 'train_loss': 4.912951151529948, 'epoch': 3.0})

In [ ]:
# Texto de entrada para comenzar la generación
input_text = "El día comenzó"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generar texto
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,  # Longitud máxima del texto generado
    num_return_sequences=1,  # Número de secuencias de texto a generar
    no_repeat_ngram_size=2,  # Tamaño del n-gram para evitar repeticiones
    temperature=1.0,  # Temperatura para la variabilidad del texto generado
    top_k=50,  # Número de tokens de alta probabilidad a considerar para cada paso
    top_p=0.95,  # Probabilidad acumulativa para seleccionar tokens de alta probabilidad
)

# Decodificar las secuencias de salida
generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in output_sequences]

# Imprimir el texto generado
for i, generated_text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}: {generated_text}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: El día comenzó a la ciudad de la vida de los más.

El cielo de una cualquierda, el ciencia de las mujeros, que el muy bueno, con el víctor.
